In [2]:
!git clone https://github.com/sahal-mulki/geo-llm

Cloning into 'geo-llm'...
remote: Enumerating objects: 49, done.
remote: Counting objects: 100% (49/49), done.
remote: Compressing objects: 100% (48/48), done.
remote: Total 49 (delta 22), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (49/49), 1.57 MiB | 3.34 MiB/s, done.


In [3]:
import pandas as pd
import json

ground_truth_yfcc, ground_truth_im = pd.read_csv("/kaggle/input/llms-for-image-geolocation-benchmarks/archive(2)/yfcc4k/yfcc4k/yfcc4k.csv"), pd.read_csv("/kaggle/input/llms-for-image-geolocation-benchmarks/archive(2)/im2gps3k_places365/im2gps3k_places365.csv")

In [4]:
geoclip_preds_yfcc, geoclip_preds_im = pd.read_csv("/kaggle/working/geo-llm/geoclip_predictions_yfcc4k.csv"), pd.read_csv("/kaggle/working/geo-llm/geoclip_predictions_im2gps3k.csv")

In [5]:
gpt4o_hinted_preds = pd.read_csv("/kaggle/input/gpt-4o-mini-responses-for-geolocation/gpt-4o-mini-geocliphints-addresses-with-coords.csv")

In [88]:
import geopy.distance

In [101]:
dict_results = []
num_of_unsuccessful = 0

for index, row in gpt4o_hinted_preds.iterrows():
    if type(row["coord_locations"]) == float:
        num_of_unsuccessful = num_of_unsuccessful + 1
        pass
    else:
        boilerplate = {
            "image_name": None,
            "dataset": None,
            "gpt_4o_coords": None,
            "gpt_4o_address": None,
            "gpt_4o_explanation": None,
            "geoclip_coords": None,
            "ground_truth": None,
            "gpt-4-truth-error": None,
            "geoclip-truth-error": None
        }
        
        boilerplate["gpt_4o_coords"] = list((float(row["coord_locations"].split(", ")[0]), float(row["coord_locations"].split(", ")[1])))
        
        boilerplate["dataset"] = row["dataset"]
        
        boilerplate["gpt_4o_explanation"] = row["response"]
        
        boilerplate["image_name"] = row["image name"]
        
        boilerplate["gpt_4o_address"] = json.loads(row["json_locations"])["final_location"]
        
        if row["dataset"] == "yfcc":
            boilerplate["geoclip_coords"] = [geoclip_preds_yfcc.iloc[geoclip_preds_yfcc.index[geoclip_preds_yfcc["Image"] == boilerplate["image_name"]]]["Prediction 1 Latitude"].values[0], geoclip_preds_yfcc.iloc[geoclip_preds_yfcc.index[geoclip_preds_yfcc["Image"] == boilerplate["image_name"]]]["Prediction 1 Longitude"].values[0]]
            boilerplate["ground_truth"] = [ground_truth_yfcc.iloc[ground_truth_yfcc.index[ground_truth_yfcc["Image"] == boilerplate["image_name"]]]["Lat"].values[0], ground_truth_yfcc.iloc[ground_truth_yfcc.index[ground_truth_yfcc["Image"] == boilerplate["image_name"]]]["Lon"].values[0]]
        else:
            boilerplate["geoclip_coords"] = [geoclip_preds_im.iloc[geoclip_preds_im.index[geoclip_preds_im["Image"] == boilerplate["image_name"]]]["Prediction 1 Latitude"].values[0], geoclip_preds_im.iloc[geoclip_preds_im.index[geoclip_preds_im["Image"] == boilerplate["image_name"]]]["Prediction 1 Longitude"].values[0]]
            boilerplate["ground_truth"] = [ground_truth_im.iloc[ground_truth_im.index[ground_truth_im["IMG_ID"] == boilerplate["image_name"]]]["LAT"].values[0], ground_truth_im.iloc[ground_truth_im.index[ground_truth_im["IMG_ID"] == boilerplate["image_name"]]]["LON"].values[0]]
        
        boilerplate["gpt-4-truth-error"] = geopy.distance.geodesic(boilerplate["gpt_4o_coords"], boilerplate["ground_truth"]).km
        
        boilerplate["geoclip-truth-error"] = geopy.distance.geodesic(boilerplate["geoclip_coords"], boilerplate["ground_truth"]).km
        
        dict_results.append(boilerplate)
        

In [97]:
dict_results[996]

{'image_name': '494341167_98a4299863_195_62126383@N00.jpg',
 'dataset': 'im2gps',
 'gpt_4o_coords': [32.0956658, 34.80625679878922],
 'gpt_4o_address': 'Tel Aviv Subdistrict, Tel-Aviv District, Israel',
 'gpt_4o_explanation': 'Based on the image of the sun setting over the ocean, I would creatively guess that this location could be **Tel Aviv Subdistrict, Tel-Aviv District, Israel**. The vibrant light and coastal scene give it a Mediterranean vibe that fits well with that area.',
 'geoclip_coords': [35.52552032470703, 11.019972801208496],
 'ground_truth': [32.347627, 34.85361],
 'gpt-4-truth-error': 28.29447270334333,
 'geoclip-truth-error': 2225.911019405124}

In [142]:
street_yfcc = []
city_yfcc = []
region_yfcc = []
country_yfcc = []
continent_yfcc = []

street_im = []
city_im = []
region_im = []
country_im = []
continent_im = []

street_yfcc_geoclip = []
city_yfcc_geoclip = []
region_yfcc_geoclip = []
country_yfcc_geoclip = []
continent_yfcc_geoclip = []

street_im_geoclip = []
city_im_geoclip = []
region_im_geoclip = []
country_im_geoclip = []
continent_im_geoclip = []

In [167]:
error_amounts_unstructured_gpt_4o_im = []
error_amounts_unstructured_geoclip_yfcc = []

error_amounts_unstructured_gpt_4o_yfcc = []
error_amounts_unstructured_geoclip_im = []

for x in dict_results:
    
    if x["dataset"] == "yfcc":
        error_amounts_unstructured_gpt_4o_yfcc.append(x["gpt-4-truth-error"])
        error_amounts_unstructured_geoclip_yfcc.append(x["geoclip-truth-error"])
        if x["gpt-4-truth-error"] <= 1:
            street_yfcc.append(x["gpt-4-truth-error"])
        elif x["gpt-4-truth-error"] <= 25:
            city_yfcc.append(x["gpt-4-truth-error"])
        elif x["gpt-4-truth-error"] <= 200:
            region_yfcc.append(x["gpt-4-truth-error"])
        elif x["gpt-4-truth-error"] <= 750:
            country_yfcc.append(x["gpt-4-truth-error"])
        else:
            continent_yfcc.append(x["gpt-4-truth-error"])
        
        if x["geoclip-truth-error"] <= 1:
            street_yfcc_geoclip.append(x["geoclip-truth-error"])
        elif x["geoclip-truth-error"] <= 25:
            city_yfcc_geoclip.append(x["geoclip-truth-error"])
        elif x["geoclip-truth-error"] <= 200:
            region_yfcc_geoclip.append(x["geoclip-truth-error"])
        elif x["geoclip-truth-error"] <= 750:
            country_yfcc_geoclip.append(x["geoclip-truth-error"])
        else:
            continent_yfcc_geoclip.append(x["geoclip-truth-error"])
    else:
        error_amounts_unstructured_gpt_4o_im.append(x["gpt-4-truth-error"])
        error_amounts_unstructured_geoclip_im.append(x["geoclip-truth-error"])
        if x["gpt-4-truth-error"] <= 1:
            street_im.append(x["gpt-4-truth-error"])
        elif x["gpt-4-truth-error"] <= 25:
            city_im.append(x["gpt-4-truth-error"])
        elif x["gpt-4-truth-error"] <= 200:
            region_im.append(x["gpt-4-truth-error"])
        elif x["gpt-4-truth-error"] <= 750:
            country_im.append(x["gpt-4-truth-error"])
        else:
            continent_im.append(x["gpt-4-truth-error"])
            
        if x["geoclip-truth-error"] <= 1:
            street_im_geoclip.append(x["geoclip-truth-error"])
        elif x["geoclip-truth-error"] <= 25:
            city_im_geoclip.append(x["geoclip-truth-error"])
        elif x["geoclip-truth-error"] <= 200:
            region_im_geoclip.append(x["geoclip-truth-error"])
        elif x["geoclip-truth-error"] <= 750:
            country_im_geoclip.append(x["geoclip-truth-error"])
        else:
            continent_im_geoclip.append(x["geoclip-truth-error"])
            
        

In [168]:
len(region_im_geoclip)

322

In [169]:
len(region_im)


386

In [170]:
len(error_amounts_unstructured_gpt_4o_im)

991

In [171]:
import statistics


In [172]:
statistics.median(error_amounts_unstructured_geoclip_im)

215.8523847854376

In [173]:
statistics.median(error_amounts_unstructured_gpt_4o_im)

175.19245866028564

In [174]:
statistics.median(error_amounts_unstructured_gpt_4o_yfcc)

522.0655051776148

In [175]:
statistics.median(error_amounts_unstructured_geoclip_yfcc)

584.6491515715724

In [176]:
def percentage(number):
    return str((len(number) / 1982) * 100) + "%"

In [177]:
print("Street Level % @ KM")
print("GEOCLIP: " + percentage(street_im_geoclip))
print("GPT-4o-mini : " + percentage(street_im))

print("\n")

print("City Level % @ KM")
print("GEOCLIP: " + percentage(city_im_geoclip))
print("GPT-4o-mini : " + percentage(city_im))

print("\n")

print("Region Level % @ KM")
print("GEOCLIP: " + percentage(region_im_geoclip))
print("GPT-4o-mini : " + percentage(region_im))

print("\n")

print("Country Level % @ KM")
print("GEOCLIP: " + percentage(country_im_geoclip))
print("GPT-4o-mini : " + percentage(country_im))

print("\n")

print("Continent Level % @ KM")
print("GEOCLIP: " + percentage(continent_im_geoclip))
print("GPT-4o-mini : " + percentage(continent_im))


Street Level % @ KM
GEOCLIP: 14.026236125126134%
GPT-4o-mini : 7.063572149344097%


City Level % @ KM
GEOCLIP: 18.970736629667005%
GPT-4o-mini : 25.02522704339051%


Region Level % @ KM
GEOCLIP: 16.246215943491425%
GPT-4o-mini : 19.475277497477293%


Country Level % @ KM
GEOCLIP: 17.961654894046415%
GPT-4o-mini : 17.558022199798184%


Continent Level % @ KM
GEOCLIP: 32.795156407669026%
GPT-4o-mini : 30.877901109989907%


In [178]:
print("Street Level % @ KM")
print("GEOCLIP: " + percentage(street_yfcc_geoclip))
print("GPT-4o-mini : " + percentage(street_yfcc))

print("\n")

print("City Level % @ KM")
print("GEOCLIP: " + percentage(city_yfcc_geoclip))
print("GPT-4o-mini : " + percentage(city_yfcc))

print("\n")

print("Region Level % @ KM")
print("GEOCLIP: " + percentage(region_yfcc_geoclip))
print("GPT-4o-mini : " + percentage(region_yfcc))

print("\n")

print("Country Level % @ KM")
print("GEOCLIP: " + percentage(country_yfcc_geoclip))
print("GPT-4o-mini : " + percentage(country_yfcc))

print("\n")

print("Continent Level % @ KM")
print("GEOCLIP: " + percentage(continent_yfcc_geoclip))
print("GPT-4o-mini : " + percentage(continent_yfcc))


Street Level % @ KM
GEOCLIP: 4.490413723511604%
GPT-4o-mini : 4.137235116044399%


City Level % @ KM
GEOCLIP: 4.339051463168516%
GPT-4o-mini : 11.806256306760847%


Region Level % @ KM
GEOCLIP: 6.912209889001009%
GPT-4o-mini : 18.163471241170534%


Country Level % @ KM
GEOCLIP: 12.4117053481332%
GPT-4o-mini : 22.906155398587284%


Continent Level % @ KM
GEOCLIP: 21.84661957618567%
GPT-4o-mini : 42.986881937436934%


In [179]:
print("Median error for GEOCLIP on YFCC 1K: " + str(statistics.median(error_amounts_unstructured_geoclip_yfcc)))
print("Median error for GEOCLIP on IM2GPS 1K: " + str(statistics.median(error_amounts_unstructured_geoclip_im)))

Median error for GEOCLIP on YFCC 1K: 584.6491515715724
Median error for GEOCLIP on IM2GPS 1K: 215.8523847854376


In [180]:
print("Median error for GPT-4o-mini on YFCC 1K: " + str(statistics.median(error_amounts_unstructured_gpt_4o_yfcc)))
print("Median error for GPT-4o-mini on IM2GPS 1K: " + str(statistics.median(error_amounts_unstructured_gpt_4o_im)))

Median error for GPT-4o-mini on YFCC 1K: 522.0655051776148
Median error for GPT-4o-mini on IM2GPS 1K: 175.19245866028564
